# Hyper parameter tunning
- updated 0921
- data  
7th_FE_ver4  
rf 튜닝
- optuna, bayesian

## y_predict 값들 피클로 저장해놓기   
stacking & scaler 구현


## ready

In [ ]:
cd /content/drive/My Drive/빅콘

/content/drive/My Drive/빅콘


In [ ]:
!apt-get update --qq
!apt-get install fonts-nanum* -qq

E: Command line option --qq is not understood in combination with the other options
Selecting previously unselected package fonts-nanum.
(Reading database ... 144676 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig 

In [ ]:
# 한글 폰트
import matplotlib.font_manager as fm
fm._rebuild()

In [ ]:
# # CUDA
# !pip install mxnet-cu80
# import mxnet as mx

In [ ]:
# ready
import os
import sys
import joblib
import pickle
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd 
import numpy as np
import random
import math

from IPython.display import display
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_info_columns', 500)

import imp
import datetime
from tqdm import tqdm

In [ ]:
# vusualize
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

In [ ]:
# sklearn
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score

## Load Data
6th_FE_ver4

data['X'] / data['y']

In [ ]:
# load data
data = joblib.load(os.path.join('data', '7th_train_FE.pkl'))
locals().update(data)

# X, y 설정
X = data['X'] ; y = data['y'] ; log_y = np.log(y)

In [ ]:
# na
X.isnull().sum().sum()

3733

In [ ]:
# 확인
#X.info()

In [ ]:
# fill na
X0 = X.fillna(0)

# One Hot
#X_dum = pd.get_dummies(X)
X.shape#, X_dum.shape

(35379, 402)

### MAPE + CV function

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
# mape
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# mape with exp
def MAPE_exp(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((np.exp(y_true) - np.exp(y_pred)) / np.exp(y_true))) * 100

# Stacking + SHAP
#### CV + Save Prediction + SHAP

In [ ]:
# stacking with cv=5
# MAPE_exp_cv 구현

def MAPE_exp_st(model, X, y, cv_splits, version, scaling=False):        
    # setting
    mape = {'val_mape' : [], 'test_mape' : [],'final_mape' : []}
    pred = {'val_idx' : [], 'val_pred' : [],
            'test_idx' : [], 'test_pred' : [],
            'final_pred' : []}      # final : test set mean 값
    
    # split train, test
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=77)
    pred['test_idx'].append(X_test.index)

    # for 문 돌리기
    cv=KFold(n_splits=cv_splits,random_state=77,shuffle=True)
    for t,v in cv.split(X_train):
        X_train_ = X_train.iloc[t]       # 훈련용
        X_val_ = X_train.iloc[v]         # 검증용 분리.
        y_train_ = y_train.iloc[t]
        y_val_ = y_train.iloc[v]

        pred['val_idx'].append(v)

        # scaling : MinMax or Standard
        if scaling : 
            if scaling == 'MinMax':
                scaler = MinMaxScaler()
            elif scaling == 'Standard':
                scaler = StandardScaler()
            # fitting
            X_train_ = scaler.fit_transform(X_train_)
            X_val_ = scaler.transform(X_val_)
            X_test = scaler.transform(X_test)
        
        else:
            X_test = X_test

        # modeling
        model.fit(X_train_,y_train_)
        y_pred = model.predict(X_val_)
        pred['val_pred'].append(np.exp(y_pred))

        test_pred = model.predict(X_test)
        pred['test_pred'].append(np.exp(test_pred))

        # mape
        mape['val_mape'].append(MAPE_exp(y_val_, y_pred))
        mape['test_mape'].append(np.exp(test_pred))

    # final values
    final_test = np.mean(pred['test_pred'], axis=0)
    final_mape = MAPE_exp(y_test, final_test)
    pred['final_pred'].append(final_test)
    mape['final_mape'].append(final_mape)
    
    # loss
    loss = - np.mean(mape)

    # save pickle
    with open('mape.pickle' + '_' + version, 'wb') as f:
        pickle.dump(mape, f, pickle.HIGHEST_PROTOCOL)
    with open('pred.pickle' + '_' + version, 'wb') as f:
        pickle.dump(pred, f, pickle.HIGHEST_PROTOCOL)

    return mape, pred, loss

## Tunning ML Basic

# tunning

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [ ]:
rf = RandomForestRegressor()
rf

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
# rf
#def MAPE_exp_st(model, X, y, cv_splits, version, scaling=False):
# stacking with cv=5
# MAPE_exp_cv 구현

def rf_model(model, X, y, cv_splits, version, scaling=False):        
    # setting
    mape = {'val_mape' : [], 'test_mape' : [],'final_mape' : []}
    pred = {'val_idx' : [], 'val_pred' : [],
            'test_idx' : [], 'test_pred' : [],
            'final_pred' : []}      # final : test set mean 값
    
    # split train, test
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=77)
    pred['test_idx'].append(X_test.index)

    # for 문 돌리기
    cv=KFold(n_splits=cv_splits,random_state=77,shuffle=True)
    for t,v in cv.split(X_train):
        X_train_ = X_train.iloc[t]       # 훈련용
        X_val_ = X_train.iloc[v]         # 검증용 분리.
        y_train_ = y_train.iloc[t]
        y_val_ = y_train.iloc[v]

        pred['val_idx'].append(v)

        # scaling : MinMax or Standard
        if scaling : 
            if scaling == 'MinMax':
                scaler = MinMaxScaler()
            elif scaling == 'Standard':
                scaler = StandardScaler()
            # fitting
            X_train_ = scaler.fit_transform(X_train_)
            X_val_ = scaler.transform(X_val_)
            X_test = scaler.transform(X_test)
        
        else:
            X_test = X_test

        # modeling
        model.fit(X_train_,y_train_)
        y_pred = model.predict(X_val_)
        pred['val_pred'].append(np.exp(y_pred))

        test_pred = model.predict(X_test)
        pred['test_pred'].append(np.exp(test_pred))

        # mape
        mape['val_mape'].append(MAPE_exp(y_val_, y_pred))
        mape['test_mape'].append(np.exp(test_pred))

    # final values
    final_test = np.mean(pred['test_pred'], axis=0)
    final_mape = MAPE_exp(y_test, final_test)
    pred['final_pred'].append(final_test)
    mape['final_mape'].append(final_mape)
    
    # loss
    loss = - np.mean(mape)


    return loss


rfBO = BayesianOptimization(rf_model(rf,X0,log_y,5,'rf_0919',scaling='MinMax'),
                             {'max_depth': (0, 1000),
                              'min_samples_split' : (2,100),
                              'min_samples_leaf' : (2,30),
                              'n_estimators' : (10,1000),
                             },
                             random_state=77)     # 77

rfBO.maximize(init_points=10, n_iter=20)

### ML + baysyan
https://data-newbie.tistory.com/159

### ML + optuna